In [1]:
import ccxt
import pandas as pd
import numpy as np

In [3]:
# Binance API bağlantısı
#### 212.253.203.202 ip adresi için btc-türk için apikey, oluşturma. 
exchange = ccxt.btcturk({
    'apiKey': 'd75b913f-c8cd-4d0b-8a60-87c802372fa4',
    'secret': 'MugC1ziP8fC3pvGzdj8W2aerQzP7zPVZ',
})




In [ ]:
# Piyasa verilerini çekme
ticker = exchange.fetch_ticker('BTC/USDT')
print(ticker)

In [ ]:
import ccxt

# BTC Türk'e bağlan
exchange = ccxt.btcturk({
    'apiKey': "b4cac2f5-596e-44e0-b1f3-69c7eb9aeb35",
    'secret': "Cr0SVA21QMtj+qh/PTI8peZujI1fP4xw",
    'enableRateLimit': True 
})


# Piyasa bilgisini al
exchange.load_markets()

# BTC/TRY için mevcut fiyatı çek
btc_try = exchange.fetch_ticker('BTC/TRY')
btc_try_price = btc_try["last"]  # Son fiyat

# 1000 TL karşılığı kaç BTC alınabilir hesapla
amount = 1000 / btc_try_price
amount

# Piyasa emri ile BTC al
# order = exchange.create_market_buy_order('BTC/TRY', amount)

# İşlem sonucunu yazdır
# print("BTC ALIM EMRİ BAŞARILI:", order)



In [ ]:
import time
import hmac
import hashlib
import base64
import requests
import json

# # BTC Türk'e bağlan
# exchange = ccxt.btcturk({
#     'apiKey': "b4cac2f5-596e-44e0-b1f3-69c7eb9aeb35",
#     'secret': "Cr0SVA21QMtj+qh/PTI8peZujI1fP4xw",
# })

# BTC Türk API Anahtarlarınız
API_KEY = "b4cac2f5-596e-44e0-b1f3-69c7eb9aeb35"
API_SECRET = "Cr0SVA21QMtj+qh/PTI8peZujI1fP4xw"

# API SECRET'ı base64 formatından çöz
api_secret_decoded = base64.b64decode(API_SECRET)

# Zaman damgası (timestamp) oluştur
stamp = str(int(time.time()) * 1000)

# Veriyi HMAC SHA256 ile imzalama
data = "{}{}".format(API_KEY, stamp).encode('utf-8')
signature = hmac.new(api_secret_decoded, data, hashlib.sha256).digest()
signature = base64.b64encode(signature).decode()

# HTTP Başlıklarını (Headers) ayarla
headers = {
    "X-PCK": API_KEY,
    "X-Stamp": stamp,
    "X-Signature": signature,
    "Content-Type": "application/json"
}

# BTC/TRY fiyatını çekme
ticker_url = "https://api.btcturk.com/api/v2/ticker"
response = requests.get(ticker_url)
ticker_data = response.json()

# BTC/TRY fiyatını bul
btc_try_price = None
for pair in ticker_data["data"]:
    if pair["pair"] == "BTCTRY":
        btc_try_price = float(pair["last"])
        break

if btc_try_price is None:
    print("BTC fiyatı alınamadı!")
    exit()

# 1000 TL karşılığı kaç BTC alınabilir?
amount = 1000 / btc_try_price
amount


In [ ]:
# Alım Emri (BUY ORDER) oluşturma
order_url = "https://api.btcturk.com/api/v1/order"

# Sipariş verileri
order_data = {
    "quantity": round(amount, 6),  # Kaç BTC alınacağı (6 ondalık basamak)
    "price": btc_try_price,  # Piyasa fiyatı
    "stopPrice": 0,  # Stop fiyat (Piyasa emri için 0 kalabilir)
    "newOrderClientId": "123456",  # Rastgele bir müşteri ID
    "orderMethod": "market",  # Piyasa emri (market order)
    "orderType": "buy",  # Alış emri
    "pairSymbol": "BTCTRY"  # İşlem çifti
}

# API'ye alım emrini gönderme
response = requests.post(order_url, headers=headers, json=order_data)

# # Yanıtı ekrana yazdır
if response.status_code == 200:
    print("BTC ALIM EMRİ BAŞARILI:", response.json())
else:
    print("HATA OLUŞTU:", response.json())